# OpenLangMem

In [4]:
# if you need it
%pip install anthropic
import uuid

Note: you may need to restart the kernel to use updated packages.


In [6]:
from anthropic import Anthropic
import json
import uuid
import getpass
import os


os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("ANTHROPIC_API_KEY")

client = Anthropic()
MODEL_NAME = "claude-3-haiku-20240307"

johnny_user_id = uuid.uuid4()
jimmy_user_id = uuid.uuid4()
jimmy_username = f"jimmy-{uuid.uuid4().hex[:4]}"
johnny_username = f"johnny-{uuid.uuid4().hex[:4]}"

ANTHROPIC_API_KEY ········


In [7]:
# Unique for a given converstaion
thread_id = uuid.uuid4()

messages = [
    {"role": "system", "content": "You are a helpful AI assistant"},
    {
        "role": "user",
        # Names are optional but should be consistent with a given user id, if provided
        "name": jimmy_username,
        "content": "Hey johnny have i ever told you about my older bro steve?",
        "metadata": {
            "user_id": str(jimmy_user_id),
        },
    },
    {
        "content": "no, you didn't, but I think he was friends with my younger sister sueann",
        "role": "user",
        "name": johnny_username,
        "metadata": {
            "user_id": str(johnny_user_id),
        },
    },
    {
        "content": "yeah me and him used to play stickball down in the park before school started. I think it was in 1980",
        "role": "user",
        "name": jimmy_username,
        "metadata": {
            "user_id": str(jimmy_user_id),
        },
    },
    {
        "content": "That was totally 1979! I remember because i was stuck at home all summer.",
        "role": "user",
        "name": "Jeanne",
        # If the user ID isn't provided, we treat this as a guest message and won't assign memories to the user
    },
    {
        "content": "That was so long ago. I have gotten old and gained 200 pounds since then. I can't even remember who was president. @ai, who was the president in 1980?",
        "role": "user",
        "name": johnny_username,
        "metadata": {
            "user_id": str(johnny_user_id),
        },
    },
    {
        "content": "The president of the United States in 1980 was Jimmy Carter.",
        "role": "assistant",
    },
    {
        "content": "Wow ya i forgot that. Stickleball really impacted my life. It's how i first met Jeanne! wonder how my life would have turned out if it hadn't happened that way.",
        "role": "user",
        "name": jimmy_username,
        "metadata": {
            "user_id": str(jimmy_user_id),
        },
    },
    {
        "content": "Yeah wow. That was a big year! @ai could you remind me what else was going on that year?",
        "role": "user",
        "name": johnny_username,
        "metadata": {
            "user_id": str(johnny_user_id),
        },
    },
]

# result = await completion(messages)

# messages.append(result.choices[0].message)
# print(result.choices[0].message.content)

# Thread Summary

In [8]:
tools = [
    {
        "name": "thread_summary",
        "description": "Prints a summary of the conversation.",
        "input_schema": {
            "type": "object",
            "properties": {
                "title": {"type": "string", "description": "Distinct for the conversation."},
                "topics": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": 'Array of tags for topics discussed in this conversation, e.g. ["tech", "politics"]. Should be as specific as possible, and can overlap.'
                },
                "summary": {"type": "string", "description": "High level summary of the interactions. One or two paragraphs max."},
                "coherence": {"type": "integer", "description": "Coherence of the conversation, 0-100 (inclusive)"},
                "persuasion": {"type": "number", "description": "Conversation's persuasion score, 0.0-1.0 (inclusive)"}
            },
            "required": ['title', 'topics', 'summary', 'coherence', 'persuasion', 'counterpoint']
        }
    }
]

def create_pretty_string(msgs):
    result_lines = []
    for msg in msgs:
        name = msg.get('name', 'Unknown User') if msg.get('role') == 'user' else 'system'
        result_lines.append(f"{name}: {msg['content']}")
    return "\n".join(result_lines)

query = f"""
<conversation>
{create_pretty_string(messages)}
</conversation>

Use the `thread_summary` tool.
"""

response = client.beta.tools.messages.create(
    model=MODEL_NAME,
    max_tokens=4096,
    tools=tools,
    messages=[{"role": "user", "content": query}]
)
json_summary = None
for content in response.content:
    if content.type == "tool_use" and content.name == "thread_summary":
        json_summary = content.input
        break

if json_summary:
    print("JSON Summary:")
    print(json.dumps(json_summary, indent=2))
else:
    print("No JSON summary found in the response.")

JSON Summary:
{
  "title": "Reminiscing About Stickball and 1980",
  "topics": [
    "nostalgia",
    "childhood memories",
    "presidential trivia"
  ],
  "summary": "The conversation starts with Jimmy-2254 mentioning his older brother Steve and how they used to play stickball in the park before school in 1980. Jeanne chimes in to correct the year, saying it was actually 1979. Johnny-a229 then comments on how long ago that was and how much he's aged since then, forgetting who the president was at the time. The system provides the answer - it was Jimmy Carter. The group continues reminiscing about how stickball impacted their lives, with Jimmy-2254 noting that it's how he first met Jeanne. They wonder how their lives would have turned out differently if that hadn't happened. Finally, Johnny-a229 asks the system to provide more context about what else was happening in 1980.</summary>\n<parameter name=\"coherence\">90",
  "persuasion": 0.7
}


# User Profile

In [9]:
tools = [
    {
        "name": "user_state",
        "description": "Prints extract named entities.",
        "input_schema": {
            "type": "object",
            "properties": {
                "preferred_name": {"type": "string", "description": "The user's name."},
                "summary": {"type": "string", "description": "A quick summary of how the user would describe themselves."},
                "interests": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": 'Array of short (two to three word) descriptions of areas of particular interest for the user. This can be a concept, activity, or idea. Favor broad interests over specific ones.'
                },
                "other_info": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": ''
                },
                "relationships": {
                    "type": "array",
                    "description": 'A list of friends, family members, colleagues, and other relationships.',
                    "items": {
                        "type": "object",
                        "description": "A person's biographical details.",
                        "properties": {
                            "name": {"type": "string", "description": "The name of the person."},
                            "relation": {"type": "string", "description": "The relation of the person to the user."},
                            "context": {"type": "string", "description": "A detailed yet concise history of things the person has done with the user."}
                        },
                        "required": ["name", "relation", "context"]
                    }
                }
            },
            "required": ["summary"]
        }
    }
]

text = "John works at Google in New York. He met with Sarah, the CEO of Acme Inc., last week in San Francisco."

query = f"""
<document>
{text}
</document>

Use the user_state tool.
"""

response = client.beta.tools.messages.create(
    model=MODEL_NAME,
    max_tokens=4096,
    tools=tools,
    messages=[{"role": "user", "content": query}]
)

json_entities = None
for content in response.content:
    if content.type == "tool_use" and content.name == "user_state":
        json_entities = content.input
        break

if json_entities:
    print("Extracted Entities (JSON):")
    print(json_entities)
else:
    print("No entities found in the response.")

Extracted Entities (JSON):
{'summary': 'John works at Google in New York. He met with Sarah, the CEO of Acme Inc., last week in San Francisco.'}


# Append State

In [17]:
tools = [
  {
      "name": "core_belief_analysis",
      "description": "Analyzes and records the core beliefs of the user as extracted from conversations.",
      "input_schema": {
        "type": "object",
        "properties": {
          "entries": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "belief": {
                  "type": "string",
                  "description": "The belief the user has about the world, themselves, or anything else."
                },
                "why": {
                  "type": "string",
                  "description": "Why the user believes this."
                },
                "context": {
                  "type": "string",
                  "description": "The raw context from the conversation that leads you to conclude that the user believes this."
                }
              },
              "required": ["belief", "why", "context"]
            }
          }
        },
        "required": ["entries"]
      }
    },
    {
      "name": "formative_event_logging",
      "description": "Logs a significant, formative event in the user's life and its impact on them. This tool appends the recorded event to the user's state, contributing to a comprehensive understanding of the user's experiences.",
      "input_schema": {
        "type": "object",
        "properties": {
          "entries": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "event": {
                  "type": "string",
                  "description": "The event that occurred. Must be important enough to be formative for the user."
                },
                "impact": {
                  "type": "string",
                  "description": "How this event influenced the user."
                }
              },
              "required": ["event", "impact"]
            }
          }
        },
        "required": ["entries"]
      }
    }
]

query = f"""
<conversation>
{create_pretty_string(messages)}
</conversation>

Use the `formative_event_logging` and `core_belief_analysis` tool
"""

response = client.beta.tools.messages.create(
    model=MODEL_NAME,
    max_tokens=4096,
    tools=tools,
    messages=[{"role": "user", "content": query}]
)
json_summary = None
for content in response.content:
    if content.type == "tool_use" and content.name in ("formative_event_logging", "core_belief_analysis") :
        json_summary = content.input
        break

if json_summary:
    print("JSON Summary:")
    print(json.dumps(json_summary, indent=2))
else:
    print("No JSON summary found in the response.")


JSON Summary:
{
  "entries": [
    {
      "event": "Playing stickball with his brother Steve in the park before school started in 1979",
      "impact": "This event led Jimmy to meet his future wife Jeanne, which significantly impacted the course of his life"
    }
  ]
}


# User Semantic Memory

In [18]:
tools = [{
  "name": "user_reflection_analysis",
  "description": "Analyzes reflections on specific activities or experiences (like stickball) and their significance to the user's life, along with scoring based on recency, importance, and relevance.",
  "input_schema": {
    "type": "object",
    "properties": {
      "entries": {
        "type": "array",
        "items": {
          "type": "object",
          "properties": {
            "content": {
              "type": "object",
              "properties": {
                "subject": {
                  "type": "string",
                  "description": "The subject of the reflection."
                },
                "predicate": {
                  "type": "string",
                  "description": "The user's reflection on the activity or experience."
                },
                "object": {
                  "type": "string",
                  "description": "The impact of the activity or experience on the user's life."
                }
              },
              "required": ["subject", "predicate", "object"]
            },
            "scores": {
              "type": "object",
              "properties": {
                "recency": {
                  "type": "number",
                  "description": "The recency score of the reflection."
                },
                "importance": {
                  "type": "number",
                  "description": "The importance score of the reflection."
                },
                "relevance": {
                  "type": "number",
                  "description": "The relevance score of the reflection."
                }
              },
              "required": ["recency", "importance", "relevance"]
            }
          },
          "required": ["content", "scores"]
        }
      }
    },
    "required": ["entries"]
  }
}]


query = f"""
<conversation>
{create_pretty_string(messages)}
</conversation>

Use the `user_reflection_analysis` tool
"""

response = client.beta.tools.messages.create(
    model=MODEL_NAME,
    max_tokens=4096,
    tools=tools,
    messages=[{"role": "user", "content": query}]
)
json_summary = None
for content in response.content:
    if content.type == "tool_use" and content.name == "user_reflection_analysis" :
        json_summary = content.input
        break

if json_summary:
    print("JSON Summary:")
    print(json.dumps(json_summary, indent=2))
else:
    print("No JSON summary found in the response.")


JSON Summary:
{
  "entries": [
    {
      "content": {
        "subject": "stickball",
        "predicate": "used to play before school started",
        "object": "impacted my life, it's how I first met Jeanne"
      },
      "scores": {
        "recency": 4,
        "importance": 5,
        "relevance": 5
      }
    }
  ]
}


# MongoDB Atlas Vector Search


In [40]:
# if you need it
# %pip install anthropic pymongo

In [41]:
from pymongo import MongoClient
# client = MongoClient("localhost", 55397) 
client = MongoClient("mongodb://localhost:55397/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.2.2&uuidRepresentation=standard")
db = client["open_lang_mem_test"]
memories = db.test_collection
import datetime
import pprint

# # https://pymongo.readthedocs.io/en/stable/tutorial.html
# post = {
#     "author": "Mike",
#     "text": "My first blog post!",
#     "tags": ["mongodb", "python", "pymongo"],
#     "date": datetime.datetime.now(tz=datetime.timezone.utc),
# }
# posts = db.posts
# post_id = posts.insert_one(post).inserted_id
# posts.find_one({"author": "Mike"})
# posts.find_one({"_id": post_id})
# posts.insert_many(new_posts)
# for post in posts.find({"author": "Mike"}):
#     pprint.pprint(post)
# posts.count_documents({"author": "Mike"})
# d = datetime.datetime(2009, 11, 12, 12)
# for post in posts.find({"date": {"$lt": d}}).sort("author"):
#     pprint.pprint(post)
# result = db.profiles.create_index([("user_id", pymongo.ASCENDING)], unique=True)
# sorted(list(db.profiles.index_information()))

In [42]:
# definition
def add_messages(_thread_id, _messages):
    memories.create_index([("content", "text")])
    for msg in _messages: # Update each message to include _thread_id in metadata
        if 'metadata' not in msg: # ensure 'metadata' exists and is a dictionary
            msg['metadata'] = {}
        msg['metadata']['thread_id'] = _thread_id
    memories.insert_many(_messages)
    
def list_messages(thread_id):
    # Query to find messages with the given thread_id in their metadata
    query = {"metadata.thread_id": thread_id}
    # Retrieve and return the list of messages
    messages = list(memories.find(query))
    return messages
def query_user_memory(**kwargs):
    user_id = kwargs.get('user_id')
    text = kwargs.get('text')
    k = kwargs.get('k', 3)  # Default to 3 if not specified
    # Construct the query to match the user_id and text search
    query = {
        # "metadata.user_id": str(user_id), # not needed - swyx
        # "$text": {"$search": text} # cant use because its not substring
        "content": {"$regex": text, "$options": "i"}  # 'i' option for case-insensitive search
    }

    results = memories.find(query).limit(k)
    # # Project the score to sort by relevance
    # projection = {
    #     "score": {"$meta": "textScore"}
    # }
    # Execute the query, sort by relevance (textScore), and limit to top k results
    # results = memories.find(query, projection).sort([("score", {"$meta": "textScore"})]).limit(k)
    return list(results)

# usage & print
add_messages(thread_id, messages)
print('--------LIST_MESSAGES--------')
for mem in list_messages(thread_id):
    pprint.pprint(mem)
    
print('--------LIST_MESSAGES--------')
print('--------QUERY__MEMORY--------')
for mem in query_user_memory(
        user_id=jimmy_user_id, text="stickleball", k=3
):
    pprint.pprint(mem)
print('--------QUERY__MEMORY--------')

# just to reset the collection
memories.delete_many({}) 


--------LIST_MESSAGES--------
{'_id': ObjectId('6611cbead515d6a6d3fae5da'),
 'content': 'You are a helpful AI assistant',
 'metadata': {'thread_id': UUID('e8f605c3-f6b7-4ab0-af3f-33194f817292')},
 'role': 'system'}
{'_id': ObjectId('6611cbead515d6a6d3fae5db'),
 'content': 'Hey johnny have i ever told you about my older bro steve?',
 'metadata': {'thread_id': UUID('e8f605c3-f6b7-4ab0-af3f-33194f817292'),
              'user_id': '6d51ac48-9e58-4666-a5b4-05f89eb9d098'},
 'name': 'jimmy-2aec',
 'role': 'user'}
{'_id': ObjectId('6611cbead515d6a6d3fae5dc'),
 'content': "no, you didn't, but I think he was friends with my younger sister "
            'sueann',
 'metadata': {'thread_id': UUID('e8f605c3-f6b7-4ab0-af3f-33194f817292'),
              'user_id': 'c9412e99-0a7e-4a0f-97fc-7f828d94fe12'},
 'name': 'johnny-0373',
 'role': 'user'}
{'_id': ObjectId('6611cbead515d6a6d3fae5dd'),
 'content': 'yeah me and him used to play stickball down in the park before '
            'school started. I thi

DeleteResult({'n': 9, 'electionId': ObjectId('7fffffff0000000000000001'), 'opTime': {'ts': Timestamp(1712444745, 10), 't': 1}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1712444745, 18), 'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'keyId': 0}}, 'operationTime': Timestamp(1712444745, 10)}, acknowledged=True)